In [1]:
%pwd

'/home/gourav/ML/PCB F D temp/research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'/home/gourav/ML/PCB F D temp'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataProcessingConfig:
    source_dir: Path
    raw_data_dir: Path
    processed_data_dir: Path
    split_data_dir: Path

In [5]:
# importing constants and utils
from src.YOLO_V8.constants import *
from src.YOLO_V8.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_data_process_config(self) -> DataProcessingConfig:
        config = self.config.data_processing 

        data_processing_config = DataProcessingConfig(
            source_dir= config.source_dir,
            raw_data_dir= config.raw_data_dir,
            processed_data_dir= config.processed_data_dir,
            split_data_dir = config.split_data_dir
        )

        return data_processing_config

In [7]:
import os
import shutil
from src.YOLO_V8 import logger
from src.YOLO_V8.utils.common import get_size

In [ ]:
def path_join(root_path, join_file):
    return os.path.join(root_path, join_file)

In [8]:
class DataProcessing:
    def __init__(self, config: DataProcessingConfig):
        self.config = config
    
    def get_raw_data(self):
        target_dir = "/home/gourav/ML/PCB F D temp/src/YOLO_V8/data/raw/"
        source_dir = "/home/gourav/ML/PCB F D temp/artifacts/data_ingestion/PCBData"
        for groups in os.listdir(source_dir):
            folder_path = os.path.join(source_dir, groups)
            for folder in os.listdir(folder_path):

                target_folder = path_join(target_dir, folder)

                for files in os.listdir(folder):
                    files_source_path = os.path.join(folder, files)
                    files_target_path = os.path.join(target_folder, files)
                    shutil.copy(files_source_path, files_target_path)

    def get_processed_images(image_folder, image_target_folder):

        for image_file in os.listdir(image_folder):
            image_file_path = path_join(image_folder, image_file)
            target_image_file_path = path_join(image_target_folder, image_file)
            shutil.copy(image_file_path, target_image_file_path)  
    
    def processed_data():
    source_folder = Path("/home/gourav/ML/PCB F D temp/src/YOLO_V8/data/raw")
    target_folder = Path("/home/gourav/ML/PCB F D temp/src/YOLO_V8/data/processed")

    if not os.path.isdir(target_folder):
        os.makedirs(target_folder)

    for folder in source_folder:
        if folder == "images":
            image_folder = os.path.join(source_folder, folder)
            image_target_folder = os.path.join(target_folder, folder)
            get_processed_images(image_folder, image_target_folder)
            

        if folder == "labels":
            pass


    def get_coordinate_line(line):
        number = line.split(" ")
        get_index = number[-1].split("\n")[0]

        number = list(map(float,number[:-1]))
        print(number)

        x_center, y_center = (number[2] + number[0]) // 2  , (number[3] + number[1]) // 2
        box_width, box_height = (number[2] - number[0]) // 2  , (number[3] - number[1]) // 2

        bbox = list(map(lambda x: x/640, [x_center, y_center, box_width, box_height]))
        print(bbox)
        line = list(map(str, bbox))
        line.insert(0,get_index)

        result_line = " ".join(line) + "\n"
        print(result_line)
        return result_line          

    def get_processed_labels(labels_folder, labels_target_folder):
        for labels_file in os.listdir(labels_folder):
            labels_file_path = path_join(labels_folder, labels_file)
            target_file_path = path_join(labels_target_folder, labels_file)
            f = open(labels_file_path, "r")
            t = open(target_file_path, "w")
            temp = []
            for lines in f.readlines():
                process_line = get_coordinate_line(lines)
                temp.append(process_line)
            
            t.writelines(temp)
            f.close()
            t.close()